In [9]:
# !pip install pyomo
# !pip install --upgrade --user ortools
# !apt-get install -y glpk-utils
# !sudo apt-get update -y
# !sudo apt-get install -y coinor-cbc

In [12]:
# !pip install -U git+https://github.com/coin-or/pulp
# !sudo pulptest
!pip install pandas

     |████████████████████████████████| 10.2 MB 9.1 MB/s 
     |████████████████████████████████| 15.6 MB 6.2 MB/s 
     |████████████████████████████████| 510 kB 7.3 MB/s 


In [10]:
import pandas as pd
from pulp import *
full_player_scores = pd.read_csv('full_players.csv', index_col=0)
full_player_scores = full_player_scores.reset_index(drop=True)
full_player_scores['Score'] = pd.to_numeric(full_player_scores['Score'], errors='coerce')
# full_player_scores['Value'] = full_player_scores['Score']/full_player_scores['Price']

full_player_scores_2021 = full_player_scores[full_player_scores['Year']==2021]
full_player_scores = full_player_scores[full_player_scores['Year']==2020]

# Only keep players who are playing this season
full_player_scores[full_player_scores['Name'].isin(full_player_scores_2021['Name'].unique())].dropna(how='all').dropna(how='all')


# full_player_scores = full_player_scores[full_player_scores.isin(full_player_scores_2021['Name'].unique())]['Name'].dropna(how='all')

player_num_games_played = full_player_scores[full_player_scores['Score']>0].groupby('Name')['Team 2'].transform('count')

opponent_average_score = full_player_scores.groupby('Team 2')['Score'].transform('mean')
player_average_score = full_player_scores.groupby('Name')['Score'].transform('mean')
player_average_value = player_average_score/full_player_scores['Price']

na_locs = full_player_scores['Score'].isna()

full_player_scores.loc[na_locs,'Score'] = ((opponent_average_score+player_average_score)/2).loc[na_locs]

# RU_Players = full_player_scores[(full_player_scores['Round']<=2)&(full_player_scores['Year']==2020)] # Only two rounds
# RU_Players = full_player_scores[(full_player_scores['Round']<=5)&(full_player_scores['Position']=='RU')&(full_player_scores['Year']==2020)] # Only rucks
# RU_Players = full_player_scores[(full_player_scores['Round']<=2)&(full_player_scores['Year']==2020)]
# RU_Players = full_player_scores[(full_player_scores['Position']=='RU')&(full_player_scores['Year']==2020)] # Only rucks

RU_Players = full_player_scores[(full_player_scores['Year']==2020)]


RU_Players = RU_Players.sort_values('Score',ascending=False)
RU_Players['id'] = "Round:"+RU_Players['Round'].astype(str)+"_Player:"+RU_Players['Name']
# RU_Players = RU_Players[RU_Players['Round']<3]

# Only take players who played more than 5 games
print(len(RU_Players['Name'].unique()))
players_to_keep = player_num_games_played[player_num_games_played>5].index.values
RU_Players = RU_Players.reindex(index=players_to_keep).dropna(how='all')
print(len(RU_Players['Name'].unique()))


# Only take players in top quater of average values (score/price)
# print(len(RU_Players['Name'].unique()))
# player_best_values = player_average_value.sort_values(ascending=False)[:int(len(player_average_value)/10)]
# RU_Players = RU_Players.reindex(index=player_best_values.index.values).dropna(how='all') 
print(len(RU_Players['Name'].unique()))


RU_Players = RU_Players.head(40)

652
501
501


In [11]:
trades_allowed = None
player_contraints = {}
prob = LpProblem("aflProblem", LpMaximize)
overall_score = LpVariable('OverallScore',0)
for p_id,score in zip(RU_Players['id'].iteritems(),RU_Players['Score'].iteritems()):
  p_id = p_id[1]
  player_contraints[p_id] = LpVariable(p_id, 0, 1, cat='Binary')

player_contraints = LpVariable.dicts("player_contraints", player_contraints, 0, 1, cat='Binary')
prob += lpSum([player_contraints[p_id[1]]*score[1] for p_id,score in zip(RU_Players['id'].iteritems(),RU_Players['Score'].iteritems())]), "Total score is maximized"

# START TRANFERS
RU_Players_prev_round = RU_Players
RU_Players_prev_round['Round_prev'] = RU_Players_prev_round['Round'] - 1

RU_transfers = RU_Players[['id','Position','Round','Name']].merge(RU_Players_prev_round[['id','Position','Round_prev','Name']],left_on=['Position','Round'],right_on=['Position','Round_prev'], suffixes = ('_prev','_next'))
RU_transfers['Transfer'] = RU_transfers['id_prev']+'->'+RU_transfers['id_next']

transfer_contraints = {}

# Define transfer itermediaries
for i,t_id in RU_transfers['Transfer'].iteritems():
  transfer_contraints[t_id] = LpVariable(t_id, 0, 1, cat='Binary')

transfer_contraints = LpVariable.dicts("transfer_contraints", transfer_contraints, 0, 1, cat='Binary')

# map rounds end to itermediary
for prev_player,trans in RU_transfers.groupby(['id_prev'])['Transfer'].apply(list).iteritems():
  prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[prev_player], "Previous player equals transfer intermediatary for "+prev_player

# map intermediary to next round
for next_player,trans in RU_transfers.groupby(['id_next'])['Transfer'].apply(list).iteritems():
  prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[next_player], "Next player equals transfer intermediatary for "+next_player

for round, transfers in RU_transfers[RU_transfers['Name_prev'] != RU_transfers['Name_next']].groupby(['Round'])['Transfer'].apply(list).iteritems():
  prob += lpSum([transfer_contraints[t_id] for t_id in transfers]) <= 4, f"Round: {round}, has less than orequal to 4 transfers"
# END TRANFERS

# START max players from each position
allowed_holds_per_position = {'DE': 8, "MI" : 10, 'RU' : 3, 'FO':9}

for (position,round), player in RU_Players[['id','Position','Round']].drop_duplicates().groupby(['Position', 'Round'])['id'].apply(list).iteritems():
  prob += lpSum([player_contraints[p_id] for p_id in player]) == allowed_holds_per_position[position], f"Position: {position}, has less than {allowed_holds_per_position[position]} in round {round}"

# END max players from each position


# START total players 
for r, player in RU_Players[['id','Round']].drop_duplicates().groupby(['Round'])['id'].apply(list).iteritems():
  print(player)
  prob += lpSum([player_contraints[p_id] for p_id in player]) == 30, f"Must have 30 players in round {r}"

# END total players 


# START money contraint
money = 13 * 10**6
for round,player in RU_Players[['id','Price','Round']].drop_duplicates().groupby(['Round'])['id'].apply(list).iteritems():
  prob += lpSum([player_contraints[p_id]*RU_Players[RU_Players['id']==p_id]['Price'].values[0] for p_id in player]) <= money, f"Round: {round}, has less than ${money}"
# END money contraint

print('done with pre-work')
solver = getSolver('COIN_CMD', maxSeconds=2000, msg=True,gapRel = 0.15, threads=100)


['Round:1_Player:Max Gawn', 'Round:1_Player:Brodie Grundy', "Round:1_Player:Reilly O'Brien"]
['Round:2_Player:Max Gawn', 'Round:2_Player:Brodie Grundy']
['Round:3_Player:Brodie Grundy']
['Round:4_Player:Max Gawn', 'Round:4_Player:Brodie Grundy']
['Round:5_Player:Max Gawn', 'Round:5_Player:Brodie Grundy', "Round:5_Player:Reilly O'Brien"]
['Round:6_Player:Max Gawn', 'Round:6_Player:Brodie Grundy']
['Round:7_Player:Max Gawn', 'Round:7_Player:Brodie Grundy', "Round:7_Player:Reilly O'Brien"]
['Round:8_Player:Max Gawn', 'Round:8_Player:Brodie Grundy', "Round:8_Player:Reilly O'Brien"]
['Round:9_Player:Max Gawn', 'Round:9_Player:Brodie Grundy']
['Round:10_Player:Max Gawn', 'Round:10_Player:Brodie Grundy', "Round:10_Player:Reilly O'Brien"]
['Round:11_Player:Brodie Grundy', "Round:11_Player:Reilly O'Brien"]
['Round:12_Player:Brodie Grundy']
['Round:13_Player:Brodie Grundy', "Round:13_Player:Reilly O'Brien"]
['Round:14_Player:Max Gawn', 'Round:14_Player:Brodie Grundy']
['Round:15_Player:Max Gawn'

In [23]:
# solver = getSolver('PULP_CBC_CMD', maxSeconds=100, msg=True,gapRel = 0.1, threads=100)
# solver = getSolver('COIN_CMD', msg=True, cuts=True)
prob.solve(pulp.PULP_CBC_CMD(msg=True, maxSeconds=10))
# prob.solve(PULP_CBC_CMD(gapRel = 0.05))
# solver = getSolver('GLPK_CMD')
# prob.solve(solver)
# pulp.COIN(maxSeconds=your_time_limit))
# prob.solve(solver)
list_solvers()

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'CPLEX_DLL',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'PULP_CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD']

In [21]:
roundplayers = {}
roundchanges = {}
for round in range(1,30):
  roundplayers[round] = []
  for n,t in player_contraints.items():
      if t.value()>0.5:
        if n[:len(f'Round:{round}_')] == f'Round:{round}_':
          roundplayers[round].append(n[len(f'Round:{round}_Player:'):])
  if not round == 1:
    roundchanges[f'{round}_out'] = set(roundplayers[round-1]).difference(set(roundplayers[round]))
    roundchanges[f'{round}_in'] = set(roundplayers[round]).difference(set(roundplayers[round-1]))


print(roundchanges)
print(roundplayers)


{'2_out': set(), '2_in': set(), '3_out': set(), '3_in': set(), '4_out': set(), '4_in': {'Max Gawn'}, '5_out': set(), '5_in': set(), '6_out': set(), '6_in': set(), '7_out': {'Max Gawn'}, '7_in': set(), '8_out': set(), '8_in': {'Max Gawn'}, '9_out': {'Max Gawn'}, '9_in': set(), '10_out': set(), '10_in': set(), '11_out': set(), '11_in': set(), '12_out': set(), '12_in': set(), '13_out': set(), '13_in': set(), '14_out': set(), '14_in': set(), '15_out': set(), '15_in': set(), '16_out': set(), '16_in': set(), '17_out': set(), '17_in': set(), '18_out': set(), '18_in': set(), '19_out': set(), '19_in': set(), '20_out': set(), '20_in': set(), '21_out': set(), '21_in': set(), '22_out': set(), '22_in': set(), '23_out': set(), '23_in': set(), '24_out': set(), '24_in': set(), '25_out': set(), '25_in': set(), '26_out': set(), '26_in': set(), '27_out': set(), '27_in': set(), '28_out': set(), '28_in': set(), '29_out': set(), '29_in': set()}
{1: [], 2: [], 3: [], 4: ['Max Gawn'], 5: ['Max Gawn'], 6: ['Ma

In [15]:
print(roundchanges)
print(roundplayers)


In [2]:
roundplayers = {1: ['Todd Goldstein', 'Rowan Marshall', 'Rory Lobb', 'Rory Laird', 'Brayden Maynard', 'Sam Docherty', 'Nick Haynes', 'Adam Saad', 'Darcy Byrne-Jones', 'Lewis Jetta', 'Adam Tomlinson', 'Steele Sidebottom', 'Dustin Martin', 'Michael Walters', 'Kyle Langford', 'Isaac Smith', 'Brett Bewley', 'Justin Westhoff', 'Lachie Neale'], 2: ['Todd Goldstein', 'Rowan Marshall', 'Rory Lobb', 'Rory Laird', 'Brayden Maynard', 'Sam Docherty', 'Nick Haynes', 'Darcy Byrne-Jones', 'Lewis Jetta', 'Adam Tomlinson', 'Steele Sidebottom', 'Patrick Dangerfield', 'Josh Dunkley', 'Michael Walters', 'Kyle Langford', 'Tom Phillips', 'Justin Westhoff', 'Lachie Neale'], 3: ['Todd Goldstein', 'Rowan Marshall', 'Rory Lobb', 'Rory Laird', 'Brayden Maynard', 'Sam Docherty', 'Nick Haynes', 'Brodie Smith', 'Darcy Byrne-Jones', 'Heath Shaw', 'Steele Sidebottom', 'Patrick Dangerfield', 'Josh Dunkley', 'Michael Walters', 'Isaac Smith', 'Justin Westhoff', 'Jye Caldwell', 'Lachie Neale'], 4: ['Todd Goldstein', 'Rowan Marshall', 'Rory Lobb', 'Lachie Whitfield', 'Rory Laird', 'Brayden Maynard', 'Sam Docherty', 'Nick Haynes', 'Brodie Smith', 'Heath Shaw', 'Patrick Dangerfield', 'Michael Walters', 'Jack Steven', 'Isaac Smith', 'Justin Westhoff', 'Josh Battle', 'Lachie Neale'], 5: ['Max Gawn', 'Todd Goldstein', 'Rowan Marshall', 'Rory Lobb', 'Lachie Whitfield', 'Rory Laird', 'Sam Docherty', 'Nick Haynes', 'Bradley Sheppard', 'Brodie Smith', 'Heath Shaw', 'Patrick Dangerfield', 'Michael Walters', 'Jack Steven', 'Isaac Smith', 'Josh Battle', 'Laitham Vandermeer', 'Hugh Greenwood'], 6: ['Max Gawn', 'Todd Goldstein', 'Rowan Marshall', 'Rory Lobb', 'Lachie Whitfield', 'Rory Laird', 'Callum Mills', 'Nick Haynes', 'Bradley Sheppard', 'James Sicily', 'Brodie Smith', 'Heath Shaw', 'Patrick Dangerfield', 'Dayne Zorko', 'Michael Walters', 'Isaac Smith', 'Laitham Vandermeer', 'Kane Farrell', 'Hugh Greenwood'], 7: ['Max Gawn', 'Rowan Marshall', 'Peter Ladhams', 'Rory Lobb', 'Lachie Whitfield', 'Rory Laird', 'Callum Mills', 'Nick Haynes', 'Bradley Sheppard', 'James Sicily', 'Brodie Smith', 'Heath Shaw', 'Patrick Dangerfield', 'Dayne Zorko', 'Dustin Martin', 'Michael Walters', 'Zac Bailey', 'Joshua Kennedy', 'Kane Farrell', 'Clayton Oliver', 'Hugh Greenwood'], 8: ['Max Gawn', 'Rowan Marshall', 'Peter Ladhams', 'Lachie Whitfield', 'Rory Laird', 'Callum Mills', 'Nick Haynes', 'James Sicily', 'Brodie Smith', 'Mark Blicavs', 'Heath Shaw', 'Patrick Dangerfield', 'Dayne Zorko', 'Dustin Martin', 'Michael Walters', 'Matthew Kennedy', 'Zac Bailey', 'Joshua Kennedy', 'Clayton Oliver', 'Paul Seedsman'], 9: ['Max Gawn', 'Rowan Marshall', 'Peter Ladhams', 'Lachie Whitfield', 'Rory Laird', 'Alex Witherden', 'Luke McDonald', 'Nick Haynes', 'Brodie Smith', 'Mark Blicavs', 'Heath Shaw', 'Ben McKay', 'Steele Sidebottom', 'Patrick Dangerfield', 'Dayne Zorko', 'Dustin Martin', 'Zac Bailey', 'Sam Powell-Pepper', 'Joshua Kennedy', 'Clayton Oliver', 'Paul Seedsman'], 10: ['Rowan Marshall', 'Peter Ladhams', 'Oscar McInerney', 'Lachie Whitfield', 'Rory Laird', 'Alex Witherden', 'Luke McDonald', 'Nick Haynes', 'Bailey Williams', 'Mark Blicavs', 'Heath Shaw', 'Ben McKay', 'Steele Sidebottom', 'Patrick Dangerfield', 'Dayne Zorko', 'Dustin Martin', 'Tom Phillips', 'Zac Bailey', 'Sam Powell-Pepper', 'Zachary Merrett', 'Clayton Oliver'], 11: ['Rowan Marshall', 'Peter Ladhams', 'Oscar McInerney', 'Rory Laird', 'Alex Witherden', 'Jack Crisp', 'Luke McDonald', 'Bradley Sheppard', 'Brodie Smith', 'Bailey Williams', 'Mark Blicavs', 'Ben McKay', 'Steele Sidebottom', 'Patrick Dangerfield', 'Dayne Zorko', 'Dustin Martin', 'Tom Hawkins', 'Tom Phillips', 'Zac Bailey', 'Gary Rohan', 'Zachary Merrett'], 12: ['Rowan Marshall', 'Oscar McInerney', 'Ben McEvoy', 'Jake Lloyd', 'Rory Laird', 'Alex Witherden', 'Jack Crisp', 'Luke McDonald', 'Bradley Sheppard', 'Bailey Williams', 'Mark Blicavs', 'Steele Sidebottom', 'Patrick Dangerfield', 'Dayne Zorko', 'Dustin Martin', 'Tom Hawkins', 'Tom Phillips', 'Jamie Elliott', 'Gary Rohan', 'Nick Blakey', 'Zachary Merrett'], 13: ['Rowan Marshall', 'Oscar McInerney', 'Ben McEvoy', 'Jake Lloyd', 'Lachie Whitfield', 'Rory Laird', 'Jack Crisp', 'Luke McDonald', 'Bradley Sheppard', 'Mark Blicavs', 'Steele Sidebottom', 'Patrick Dangerfield', 'Dustin Martin', 'Tom Hawkins', 'Tom Phillips', 'Jack Newnes', 'Jamie Elliott', 'Lachlan Schultz', 'Nick Blakey', 'Zachary Merrett'], 14: ['Max Gawn', 'Rowan Marshall', 'Ben McEvoy', 'Jake Lloyd', 'Lachie Whitfield', 'Jack Crisp', 'Luke McDonald', 'Bradley Sheppard', 'Liam Duggan', 'Mark Blicavs', 'Patrick Dangerfield', 'Kyle Langford', 'Tom Phillips', 'Jack Newnes', 'Jamie Elliott', 'Lachlan Schultz', 'Nick Blakey', 'Tom Scully', 'Zachary Merrett'], 15: ['Max Gawn', 'Rory Lobb', 'Ben McEvoy', 'Jake Lloyd', 'Lachie Whitfield', 'Rory Laird', 'Jack Crisp', 'Bradley Sheppard', 'Liam Duggan', 'Daniel Rich', 'Dayne Zorko', 'Kyle Langford', 'Tom Phillips', 'Jack Newnes', 'Jamie Elliott', 'Lachlan Schultz', 'Nick Blakey', 'Tom Scully', 'Zachary Merrett'], 16: ['Max Gawn', 'Rowan Marshall', 'Rory Lobb', 'Jake Lloyd', 'Lachie Whitfield', 'Rory Laird', 'Thomas Stewart', 'Bradley Sheppard', 'Liam Duggan', 'Daniel Rich', 'Mark Blicavs', 'Patrick Dangerfield', 'Dayne Zorko', 'Kyle Langford', 'Jack Newnes', 'Lachlan Schultz', 'Nick Blakey', 'Tom Scully', 'Zachary Merrett'], 17: ['Rowan Marshall', 'Rory Lobb', 'Esava Ratugolea', 'Jake Lloyd', 'Rory Laird', 'Thomas Stewart', 'Bradley Sheppard', 'Liam Duggan', 'Daniel Rich', 'Mark Blicavs', 'Jake Kolodjashnij', 'Patrick Dangerfield', 'Dayne Zorko', 'Dustin Martin', 'Kyle Langford', 'Jack Newnes', 'Lachlan Schultz', 'Zachary Merrett', 'Cameron Guthrie'], 18: ['Rowan Marshall', 'Rory Lobb', 'Esava Ratugolea', 'Rory Laird', 'Thomas Stewart', 'Jayden Short', 'Bradley Sheppard', 'Liam Duggan', 'Daniel Rich', 'Mark Blicavs', 'Jake Kolodjashnij', 'Patrick Dangerfield', 'Dayne Zorko', 'Dustin Martin', 'Jack Steven', 'Kyle Langford', 'Kane Lambert', 'Cameron Guthrie', 'Sam Menegola'], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: []}


In [3]:
roundplayers[1]

['Todd Goldstein',
 'Rowan Marshall',
 'Rory Lobb',
 'Rory Laird',
 'Brayden Maynard',
 'Sam Docherty',
 'Nick Haynes',
 'Adam Saad',
 'Darcy Byrne-Jones',
 'Lewis Jetta',
 'Adam Tomlinson',
 'Steele Sidebottom',
 'Dustin Martin',
 'Michael Walters',
 'Kyle Langford',
 'Isaac Smith',
 'Brett Bewley',
 'Justin Westhoff',
 'Lachie Neale']